In [1]:
VERSION = '99_v1_03_01'

In [2]:
model_list = [    
    {'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':1}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':1}, #deberta-base, seed100, split00_v2_07
    
    #{'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':1}, #longformer-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v1_05/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_22/oof_df.csv', 'w':1}, #deberta-v2-xlarge, seed100, split00_v2_07
    
    #{'path':'../../19_RNN/exp/result/19_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, lstm
    
    #{'path':'../../20_WoSpanDet/exp/result/20_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, huggingface pre
    #{'path':'../../20_WoSpanDet/exp/result/20_v1_02/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07, huggingface pre
    
    #{'path':'../../22_BaselineMLM/exp/result/22_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, MLM pre
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5999


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,Ineffective_1,Adequate_1,Effective_1,Ineffective_2,Adequate_2,Effective_2,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.010725,0.299656,0.022952,1,0.106507,0.023831,0.300446,0.009056,0.002334,0.270700,0.060300,0.036890,0.870802,0.092308
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.034250,0.295810,0.003274,1,0.119425,0.018766,0.308643,0.005924,0.023707,0.296091,0.013536,0.076722,0.900544,0.022734
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.091020,0.240893,0.001420,1,0.324790,0.070065,0.260808,0.002460,0.024279,0.296698,0.012356,0.185364,0.798400,0.016236
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.069588,0.260059,0.003686,1,0.248233,0.065734,0.263812,0.003787,0.085017,0.239872,0.008444,0.220339,0.763743,0.015917
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.104945,0.227082,0.001307,1,0.383832,0.139432,0.192425,0.001476,0.055173,0.268754,0.009406,0.299551,0.688261,0.012188
